## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [2]:
case  = 'wat'

if case=='atm':
    theta  = 0
    n      = 1.00029
    c      = 299792458/n
    h      = 25000
    L      = 100
    v      = n * c
    times  = linspace(1e-6,1e-4,300000,endpoint=True)
    
elif case=='wat':
    n      = 1.33
    c      = 299792458/n
    h      = 4.5
    D      = 7.3   #--------------------- diameter of tank
    R      = 7.3/2 #--------------------- radius of tank
    v      = n * c
    times  = linspace(1e-10,1e-7,200000)
    
    c1     = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
    c2     = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
    c3     = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
    c4     = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
    x1,y1  = (-0.5,-0.5) #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
    theta  = 165   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
    phi    =  10   #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM
    theta  = deg2rad(theta) #-------------------------------- Convert zenith to radians
    phi    = deg2rad(phi  ) #-------------------------------- Convert azimuth to radians

    x2     = x1 + h * tan(theta) * cos(phi) # ------------------- Calculate bottom x coordinate
    y2     = y1 + h * tan(theta) * sin(phi) #-------------------- Calculate bottom y coordinate
    cp     = (x2,y2,0)
    print('A = {}'.format((x1,y1,h)))
    print('B = {}'.format(cp))

    d1     = distance.euclidean(cp,c1)  #------------------------ Calculate the distances of the point 
    d2     = distance.euclidean(cp,c2)  #------------------------ where the particle hits the bottom of 
    d3     = distance.euclidean(cp,c3)  #------------------------ the tank from the 4 PMTs
    d4     = distance.euclidean(cp,c4)
    theta2    = pi - theta
    del theta
    theta     = theta2
    print('Particle distances from PMTs    : {} m,   {} m,   {} m,   {} m'.format( round(d1,3),round(d2,3),round(d3,3),round(d4,3)))

A = (-0.5, -0.5, 4.5)
B = (-1.6874529895378847, -0.7093800003784556, 0)
Particle distances from PMTs    : 1.83 m,   2.434 m,   1.174 m,   3.608 m


## Computations 

In [94]:
%matplotlib

plotme    = 'b vs z'
distances = [d1,d2,d3,d4]
colors    = ['k','r','b','g']
detectors = [1,2,3,4]
labels    = ['Central Detector', "Detector 2",'Detector 3','Detector 4']

for ds, color, detector,mylabel in zip(distances, colors, detectors, labels):
    
    print("==============================================")
    print("Processing detector %s"%detector)
    
    T, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], []
    L      = ds
    num_zc = -sqrt(-c**4 * L**2 * cos(theta)**2 + c**2 * L**2 * v**2 * cos(theta)**2) + c**2*L*sin(theta) - L*v**2*sin(theta)
    den    = (c**2-v**2)
    zc     = (num_zc/den)
    zc_act = zc * cos(theta)
    print('z_C : {} m'.format(round(zc_act,3)))

    for iii,t in enumerate(times):
        z        = h/cos(theta) - v*t
        k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
        t_shower = t
        t_light  = k/c
        t_total  = t_shower + t_light
        tt       = t_total

        num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
        num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
        zp       = ((num1_z - num2_z) / den)
        zm       = ((num1_z + num2_z) / den)
        v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
        v_B      = num2_z
        vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
        vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
        kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
        km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
        alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
        alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
        phip     = 90+theta-rad2deg(alphap)
        phim     = 90+theta-rad2deg(alpham)   
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)
        omegap   = (vtp / kp)
        omegam   = (vtm / km)
        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))
        
        ZM.append(zm*cos(theta))
        ZP.append(zp*cos(theta))
        BM.append(bm)            
        BP.append(bp)
        PHIP.append(phip)
        PHIM.append(phim)
        T.append(tt)
        
    K = sqrt( L**2 + zc**2 - 2*zc*L*cos(pi/2-theta))
    ALPHA   = (arccos( (zc*zc + K*K - L*L) / (2*zc*K) ))
    PHI     = 90+theta-rad2deg(ALPHA)
    #print(K, (zc*zc + K*K - L*L), (2*zc*K))
    #print("theta_C :",round(PHI))

    ZP, ZM, T, BP, BM, PHIP, PHIM = array(ZP), array(ZM), array(T), array(BP), array(BM), array(PHIP), array(PHIM)
    Tn = T*1e+9 ; TT = (Tn-Tn.min())
    hp = ZP
    hm = ZM
    cond = ( ((ZP>=0) & (ZP<=h)) | ((ZM>=0) & (ZM<=h))) # Improve further if possible
    TT2   = TT[cond] ; BM2 = BM[cond] ; BP2 = BP[cond] ; PHIP2 = PHIP[cond] ; PHIM2 = PHIM[cond] ; ZP2=ZP[cond] ; ZM2=ZM[cond]
    
    pluslen = ZP
    

    if detector==1:
        den_bright = BP2[-1]
    BM2 = BM2 / den_bright
    BP2 = BP2 / den_bright
    
    if plotme == 'b vs phi': # Use times  = linspace(1e-12,1e-7,2000000)
        plt.plot(PHIP2, BP2, c=color, ls='-', label=mylabel)
        plt.plot(PHIM2, BM2, c=color, ls='--')
        plt.axhline(1,c='k',ls=':')
        plt.axvline(PHI,c='k',ls=':')
        plt.text(PHI-3, 2, r'$\theta_C$ = {}'.format(round(PHI,2))     , fontsize=18)
        plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        plt.xlim(0,91)

    if plotme == 'b vs z': # Use times  = linspace(1e-12,1e-7,2000000)
        plt.plot(ZP2, BP2, c=color, ls='-', label=mylabel)
        plt.plot(ZM2, BM2, c=color, ls='--')
        plt.axvline(zc_act,c='k',ls=':')
        if (0<=zc_act<=h):
            plt.text(zc_act+0.03, 0.5, r'$z_C$ = {}'.format(round(zc_act,2))     , fontsize=18)
        plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        #plt.xlim(0,h)

    elif plotme == 'b vs t': # Use times  = linspace(1e-10,1e-7,300000)
        plt.plot(TT2, BP2, c=color, ls='-' , label=mylabel)
        plt.plot(TT2, BM2, c=color, ls='--')
        plt.axhline(1, c='k', ls=':')
        plt.axvline(0, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        
    elif plotme == 'phi vs t':
        plt.plot(TT2, PHIP2, c=color, ls='-' , label=mylabel)
        plt.plot(TT2, PHIM2, c=color, ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.axhline(PHI, c='k', ls=':')
        plt.axvline(0, c='k', ls=':')
        plt.text(2, PHI+1.5, r'$\theta_C = 48.75^{\circ}$' , fontsize=15) #%(PHI)
        plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(-0.1,91)
        
    elif plotme == 'z vs t':
        plt.plot(TT2, ZP2, c=color, ls='-' , label=mylabel)
        plt.plot(TT2, ZM2, c=color, ls='--')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.axhline(zc_act, c='k', ls=':')
        plt.axvline(0, c='k', ls=':')
        if (0<=zc_act<=h):
            plt.text(2, zc_act+0.05, r'$z_C = {} m$'.format(round(zc_act,2)) , fontsize=15) #%(PHI)
        plt.ylabel(r'image heights $(m)$'     , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(0,h)
            
    #del BM, BP, TT, PHIM, PHIP, ZM, ZP, T, K, PHI, ALPHA, Tn, bm, bp, zp, zm, vp, vm,
    #vtp, vtm, omegap, omegam, phip, phim, alphap, alpham, kp, km
    
plt.legend(loc='upper right', prop={'size': 14})
plt.show()

Using matplotlib backend: Qt5Agg
Processing detector 1
z_C : 2.405 m
Processing detector 2
z_C : 3.198 m
Processing detector 3
z_C : 1.543 m
Processing detector 4
z_C : 4.741 m


In [95]:
ZPn = ZP
ZPn

array([  5.02181927,   5.02197613,   5.02213299, ..., 180.72428438,
       180.72530131, 180.72631823])

In [96]:
cond = ( ((hp>=0) & (hp<=h)) | ((hm>=0) & (hm<=h)))
cond

array([ True,  True,  True, ..., False, False, False])

In [110]:
ZM

array([  4.47104227,   4.47089763,   4.47075298, ..., -24.45743848,
       -24.45758313, -24.45772777])

In [111]:
ZP

array([  5.02181927,   5.02197613,   5.02213299, ..., 180.72428438,
       180.72530131, 180.72631823])

In [113]:
condA = (0<=i<=h)

In [118]:
kk =ZP[(0<=ZP<=h)]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [129]:
c = np.array([1,2,3,4,5,6,7,8,9,10])
if len ( c[(c>=3) & (c<=7)]) == 0:
    print('nops')

In [130]:
c[(c>=3) & (c<=7)]

array([3, 4, 5, 6, 7])

In [128]:
i

16.20583716751392

In [124]:
c

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [117]:
if kk.size==0:
    print('pop')

AttributeError: module 'numpy' has no attribute 'isempty'

In [107]:
for i in ZP:
    if 

5.021819270901287
5.021976128287875
5.022132992620369
5.02228986389895
5.022446742124294
5.022603627296643
5.022760519416691
5.0229174184846395
5.023074324501145
5.023231237466498
5.023388157381124
5.023545084245681
5.023702018060453
5.023858958825952
5.024015906542623
5.024172861210655
5.024329822830983
5.024486791403764
5.024643766929267
5.024800749408396
5.024957738841167
5.025114735228266
5.025271738570106
5.0254287488671645
5.02558576611977
5.025742790328511
5.025899821493716
5.026056859615863
5.026213904695472
5.026370956732905
5.026528015728533
5.026685081683147
5.026842154596742
5.026999234470088
5.02715632130351
5.027313415097556
5.027470515852431
5.027627623568878
5.0277847382471
5.027941859887743
5.028098988491094
5.028256124057847
5.028413266588138
5.028570416082511
5.02872757254155
5.028884735965559
5.029041906355047
5.0291990837103855
5.029356268032223
5.02951345932088
5.029670657576685
5.02982786280037
5.029985074992006
5.030142294152387
5.030299520281727
5.0304567533806

In [89]:
c1 = ((ZP>=0) & (ZP<=h))
ZP = ZP[c1]
c2 = ((ZM>=0) & (ZM<=h))
ZP = ZP[c2]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 200000

In [29]:
def entry_brightness(h,c,v,theta=0,L=0):
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 0
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    den      = (c**2-v**2)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
    vtp      = vp * sin(alphap)
    vtm      = vm * sin(alpham)
    omegap   = (vtp / kp)
    omegam   = (vtm / km)
    bp       = abs(omegap / (kp**2))
    bm       = abs(omegam / (km**2))
    return bm,bp

entry_brightness(h=h,c=c,v=v,theta=0,L=0)

NameError: name 'zp' is not defined